# **TP Intelligence Artificielle - SAT et CSP**

# **Partie 0 : Visualisation**

Voici une fonction pour visualiser les sudokus.

In [1]:
from IPython.display import HTML

def visualize_sudoku(grid):
    html = "<table>"
    for i in range(9):
        html += "<tr>"
        for j in range(9):
            value = grid[i][j]
            cell_style = "width: 30px; height: 30px; text-align: center; font-size: 20px; border: 1px solid black;"
            if i % 3 == 0 and i != 0:  # Horizontal lines
                cell_style += "border-top: 5px solid black;"
            if j % 3 == 0 and j != 0:  # Vertical lines
                cell_style += "border-left: 5px solid black;"

            if value == 0:
                html += f"<td style='background-color: white; {cell_style}'> </td>"  # Case vide
            else:
                html += f"<td style='background-color: rgb(226, 114, 91); {cell_style}'>{value}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### **Exercice 1**
Écrivez un Sudoku sous forme d’une liste de listes (9 listes de 9 élements chacune où la valeur 0 represente une caise vide), puis utilisez la fonction visualize_sudoku pour l’afficher.

In [2]:
shudu = [
    [0, 0, 5, 7, 1, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [2, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 0, 2, 0, 7, 0, 6],
    [0, 0, 0, 0, 9, 0, 1, 0, 0],
    [3, 0, 0, 0, 0, 0, 2, 5, 0],
    [0, 6, 4, 5, 0, 1, 0, 0, 7],
    [8, 1, 0, 0, 0, 3, 0, 0, 5],
    [0, 0, 0, 0, 0, 2, 0, 4, 0]
]

visualize_sudoku(shudu)

,,5,7,1,9,,,
,,,,,,,,
2,,,,,,,,
,5,,,2,,7,,6
,,,,9,,1,,
3,,,,,,2,5,
,6,4,5,,1,,,7
8,1,,,,3,,,5
,,,,,2,,4,


# **Partie 1 : Modélisation**

Nous allons modéliser le Sudoku comme un problème SAT et CSP, puis utiliser les méthodes propres à chaque modélisation pour la résolution.
Pour cela, nous utiliserons deux bibliothèques : **python-sat** et **python-constraint**.

Exécutez le bloc de code suivant pour les installer.
Pour les installer sur vos PC, il suffit d’exécuter les mêmes lignes (sans le '!' au début) dans une console de commandes (CMD).



In [3]:
# Installation des libraries
# pip install python-sat
# pip install python-constraint

## **1.1 Modélisation SAT**

Pour modéliser le Sudoku comme un problème SAT, nous utiliserons la méthode CNF de la bibliothèque pysat.formula.
CNF permet de définir les clauses logiques représentant les contraintes du problème.
Pour les variables, nous utiliserons un dictionnaire.

In [4]:
from pysat.formula import CNF

class Sudoku_SAT:
    def __init__(self, grid):
        self.grid = grid
        self.cnf = CNF()  # Crée un objet CNF pour stocker les clauses
        self.var_map = {}  # Dictionnaire pour mapper (ligne, colonne, valeur) à un ID de variable SAT

        # 1. Créer les variables SAT
        var_count = 1
        for i in range(9):
            for j in range(9):
                for k in range(1, 10):
                    self.var_map[(i, j, k)] = var_count
                    var_count += 1

        # 2. Créer les clauses SAT

        # a. Chaque case doit contenir au moins une valeur
        for i in range(9):
            for j in range(9):
                clause = [self.var_map[(i, j, k)] for k in range(1, 10)]
                self.cnf.append(clause)

        # b. Chaque case ne peut contenir qu'une seule valeur
        for i in range(9):
            for j in range(9):
                for k1 in range(1, 10):
                    for k2 in range(k1 + 1, 10):
                        clause = [-self.var_map[(i, j, k1)], -self.var_map[(i, j, k2)]]
                        self.cnf.append(clause)

        # c. Chaque ligne doit contenir tous les chiffres de 1 à 9
        for i in range(9):
            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for j in range(9)]
                self.cnf.append(clause)

        # d. Chaque colonne doit contenir tous les chiffres de 1 à 9
        for j in range(9):
            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for i in range(9)]
                self.cnf.append(clause)

        # e. Chaque sous-grille 3x3 doit contenir tous les chiffres de 1 à 9
        for subgrid_row in range(3):
            for subgrid_col in range(3):
                for k in range(1, 10):
                    clause = [self.var_map[(subgrid_row * 3 + i, subgrid_col * 3 + j, k)]
                              for i in range(3) for j in range(3)]
                    self.cnf.append(clause)

        # f. Contraintes initiales de la grille
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] != 0:
                    self.cnf.append([self.var_map[(i, j, self.grid[i][j])]])

### **Exercice 2**
Créez une fonction qui prend une grille de Sudoku en entrée et utilise le solveur Glucose3 de la bibliothèque pysat.solver pour résoudre le jeu (pensez à bien importer Glucosa3). Votre fonction doit :

1. Créer un objet sudoku_sat pour modéliser le problème.
2. Ajouter au solveur les contraintes du Sudoku sous forme de clauses logiques.
3. Vérifier si le Sudoku est résoluble en utilisant le solveur SAT.
4. Trouver une solution si le Sudoku est résoluble.
5. Interpréter la solution retournée par le solveur pour remplir la grille :
  * La solution sera une liste de taille 9 × 9 × 10, contenant des valeurs positives et négatives.
  * Une valeur positive signifie que la variable (i, j, k) est True, ce qui indique que la case (i, j) doit contenir la valeur k.

In [5]:
from pysat.solvers import Glucose3

def solve_sudoku_sat(grid):
    """
    使用 SAT 求解器 (Glucose3) 解决数独问题。
    """
    # 1. 创建 sudoku_sat 对象来建模问题 (生成 CNF 子句)
    model = Sudoku_SAT(grid)
    
    # 2. 初始化求解器
    # 使用 'with' 语句可以确保求解器在使用后正确关闭释放资源
    with Glucose3() as solver:
        # 3. 将数独的约束 (CNF 子句) 添加到求解器中
        solver.append_formula(model.cnf.clauses)
        
        # 4. 检查数独是否可解
        if solver.solve():
            # 如果可解，获取模型 (即让所有子句为真的变量赋值列表)
            solution = solver.get_model()
            solution_set = set(solution)  # 转换为集合以加快查找速度
            
            # 5. 解释解决方案并填充网格
            # 遍历我们在 Sudoku_SAT 中创建的映射: (行, 列, 值) -> var_id
            for (i, j, k), var_id in model.var_map.items():
                # 如果 var_id 在解决方案中 (即该变量为 True)
                # 意味着在第 i 行第 j 列填入数字 k
                if var_id in solution_set:
                    grid[i][j] = k
            
            return grid
        else:
            print("Pas de solution trouvée (无解)")
            return None

### **Exercice 3**
Utilisez votre fonction pour resoudre le sudoku ci-dessous:

    [0, 0, 0, 0, 7, 0, 0, 0, 0]
    [6, 0, 0, 1, 9, 5, 0, 0, 0]
    [0, 9, 8, 0, 0, 0, 0, 6, 0]
    [8, 0, 0, 0, 6, 0, 0, 0, 3]
    [4, 0, 0, 8, 0, 3, 0, 0, 1]
    [7, 0, 0, 0, 2, 0, 0, 0, 6]
    [0, 6, 0, 0, 0, 0, 2, 8, 0]
    [0, 0, 0, 4, 1, 9, 0, 0, 5]
    [0, 0, 0, 0, 8, 0, 0, 7, 9]

In [6]:
probleme = [
    [0, 0, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

visualize_sudoku(solve_sudoku_sat(probleme))

5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


## **1.2 Modélisation CSP**

### **Exercice 4**

Créez une fonction solve_sudoku_csp qui prend une grille de Sudoku en entrée, la modélise comme un problème CSP et la résout. Votre fonction doit :
1. Créer une instance du problème CSP
  * Utilisez la classe Problem de la bibliothèque python-constraint pour représenter le Sudoku.
2. Définir les variables du problème
  * Utilisez la méthode addVariable pour ajouter les variables au problème, sachant que :
    * Chaque case de la grille est une variable identifiée par un tuple (ligne, colonne).
    * Le domaine de chaque variable correspond aux valeurs possibles (de 1 à 9).
    * Si une case contient déjà un chiffre (non nul), son domaine est restreint à cette valeur unique.
3. Définir les contraintes du Sudoku (les valeurs doivent être uniques dans chaque ligne, colonne et région 3×3). Pour cela, utilisez la méthode addConstraint avec AllDifferentConstraint() pour assurer ces contraintes.
4. Résoudre le problème. Pour cela utilisez la méthode getSolutions() pour obtenir toutes les solutions possibles.
5. Gérer les solutions
  * Si des solutions existent, les stocker dans une liste.
  * Sinon, indiquer qu’aucune solution n’a été trouvée.

In [7]:
from constraint import Problem, AllDifferentConstraint

def solve_sudoku_csp(grid):
    """
    使用 CSP (约束满足问题) 求解器解决数独。
    """
    # 1. 创建 CSP 问题实例
    problem = Problem()
    
    # 2. 定义变量：每个格子 (i, j) 是一个变量
    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0:
                # 如果是空格，可能的取值范围是 1 到 9
                domain = list(range(1, 10))
            else:
                # 如果已经有数字，取值范围只能是该数字（单元素列表）
                domain = [grid[i][j]]
            
            # 添加变量：变量名为 (i, j)，取值范围为 domain
            problem.addVariable((i, j), domain)
    
    # 3. 添加约束：行、列、3x3 宫格内数字必须互不相同
    
    # a. 行约束：每一行的 9 个变量必须互不相同
    for i in range(9):
        problem.addConstraint(AllDifferentConstraint(), [(i, j) for j in range(9)])
        
    # b. 列约束：每一列的 9 个变量必须互不相同
    for j in range(9):
        problem.addConstraint(AllDifferentConstraint(), [(i, j) for i in range(9)])
        
    # c. 3x3 宫格约束：每个 3x3 子网格内的 9 个变量必须互不相同
    for row_idx in range(0, 9, 3):       # 0, 3, 6
        for col_idx in range(0, 9, 3):   # 0, 3, 6
            # 收集该 3x3 区域内的所有变量
            box_vars = []
            for i in range(3):
                for j in range(3):
                    box_vars.append((row_idx + i, col_idx + j))
            problem.addConstraint(AllDifferentConstraint(), box_vars)
    
     # 4. 获取解决方案
    solutions = problem.getSolutions()
    
    if solutions:
        # 取第一个解（因为这是一个列表，里面包含所有可能的解）
        first_solution_dict = solutions[0]
        
        # 创建一个新的空网格来存放结果
        result_grid = [[0 for _ in range(9)] for _ in range(9)]
        
        # 遍历字典，把值填回去
        for (i, j), value in first_solution_dict.items():
            result_grid[i][j] = value
            
        return result_grid  # 现在返回的是你熟悉的数独列表
    else:
        print("Pas de solution trouvée")
        return None

### **Exercice 5**
Utilisez votre fonction pour resoudre le jeu suivant.

In [8]:
# Grille de Sudoku (0 représente une case vide)
grid = [[5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [21]:
visualize_sudoku(solve_sudoku_csp(grid))

5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


# **Partie 2 : Comparaison CSP vs SAT**

Nous allons comparer les deux solveurs sur 10 sudokus differents. Pour cela, voici 10 grilles.

In [10]:
grid1 = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

In [11]:
grid2 = [
    [0, 0, 3, 0, 2, 0, 6, 0, 0],
    [9, 0, 0, 3, 0, 5, 0, 0, 1],
    [0, 0, 1, 8, 0, 6, 4, 0, 0],
    [0, 0, 8, 1, 0, 2, 9, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 8],
    [0, 0, 6, 7, 0, 8, 2, 0, 0],
    [0, 0, 2, 6, 0, 9, 5, 0, 0],
    [8, 0, 0, 2, 0, 3, 0, 0, 9],
    [0, 0, 5, 0, 1, 0, 3, 0, 0]
]

In [12]:
grid3 = [
    [0, 2, 0, 6, 0, 8, 0, 0, 0],
    [5, 8, 0, 0, 0, 9, 7, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [3, 7, 0, 0, 0, 0, 5, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 8, 0, 0, 0, 0, 1, 3],
    [0, 0, 0, 0, 2, 0, 0, 0, 0],
    [0, 0, 9, 8, 0, 0, 0, 3, 6],
    [0, 0, 0, 3, 0, 6, 0, 9, 0]
]

In [13]:
grid4 = [
    [0, 0, 5, 3, 0, 0, 0, 0, 0],
    [8, 0, 0, 0, 0, 0, 0, 2, 0],
    [0, 7, 0, 0, 1, 0, 5, 0, 0],
    [4, 0, 0, 0, 0, 5, 3, 0, 0],
    [0, 1, 0, 0, 7, 0, 0, 0, 6],
    [0, 0, 3, 2, 0, 0, 0, 8, 0],
    [0, 6, 0, 5, 0, 0, 0, 0, 9],
    [0, 0, 4, 0, 0, 0, 0, 3, 0],
    [0, 0, 0, 0, 0, 9, 7, 0, 0]
]

In [14]:
grid5 = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

In [15]:
grid6 = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0]
]

In [16]:
grid7 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0]
]

In [17]:
grid8 = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 8, 5],
    [0, 0, 1, 0, 2, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 7, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 1, 0, 0],
    [0, 9, 0, 0, 0, 0, 0, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 7, 3],
    [0, 0, 2, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 9]
]

In [18]:
grid9 = [
    [0, 0, 0, 2, 6, 0, 7, 0, 1],
    [6, 8, 0, 0, 7, 0, 0, 9, 0],
    [1, 9, 0, 0, 0, 4, 5, 0, 0],
    [8, 2, 0, 1, 0, 0, 0, 4, 0],
    [0, 0, 4, 6, 0, 2, 9, 0, 0],
    [0, 5, 0, 0, 0, 3, 0, 2, 8],
    [0, 0, 9, 3, 0, 0, 0, 7, 4],
    [0, 4, 0, 0, 5, 0, 0, 3, 6],
    [7, 0, 3, 0, 1, 8, 0, 0, 0]
]

In [19]:
grid10 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0]
]

### **Exercice 6**

Créez une fonction comparer_solveurs qui prend comme entrée une liste de grilles, résout tous les jeux avec les deux solveurs, enregistre le temps d'exécution à chaque fois, et retourne les résultats sous forme de DataFrame.

Utilisez cette fonction pour résoudre tous les jeux et affichez ensuite les résultats.

In [22]:
import time
import pandas as pd
import copy

def comparer_solveurs(grids):
    """
    比较 SAT 和 CSP 求解器在给定数独列表上的性能。
    返回一个包含执行时间的 Pandas DataFrame。
    """
    results = []

    for idx, grid in enumerate(grids):
        # 为了不修改原始 grid，我们需要深拷贝
        grid_sat = copy.deepcopy(grid)
        grid_csp = copy.deepcopy(grid)
        
        # --- 测试 SAT 求解器 ---
        start_time = time.time()
        solve_sudoku_sat(grid_sat)
        end_time = time.time()
        sat_time = end_time - start_time
        
        # --- 测试 CSP 求解器 ---
        start_time = time.time()
        solve_sudoku_csp(grid_csp)
        end_time = time.time()
        csp_time = end_time - start_time
        
        # 记录结果
        results.append({
            "Grid ID": idx + 1,
            "SAT Time (s)": sat_time,
            "CSP Time (s)": csp_time
        })
    
    # 创建 DataFrame
    df = pd.DataFrame(results)
    return df

# 定义要测试的网格列表
grids_to_test = [grid1, grid2, grid3, grid4, grid5, grid6, grid7, grid8, grid9, grid10]

# 运行比较
df_results = comparer_solveurs(grids_to_test)

In [23]:
from tabulate import tabulate
print(tabulate(df_results, headers = 'keys', tablefmt = 'pretty'))

+---+---------+----------------------+----------------------+
|   | Grid ID |     SAT Time (s)     |     CSP Time (s)     |
+---+---------+----------------------+----------------------+
| 0 |   1.0   | 0.01629161834716797  | 0.004551410675048828 |
| 1 |   2.0   | 0.013778448104858398 | 0.004812479019165039 |
| 2 |   3.0   | 0.05898118019104004  | 0.13806915283203125  |
| 3 |   4.0   |  0.2772338390350342  | 0.06798076629638672  |
| 4 |   5.0   | 0.33717942237854004  |  0.8607032299041748  |
| 5 |   6.0   |  0.2561070919036865  |  0.3861720561981201  |
| 6 |   7.0   | 0.24338459968566895  |  0.6980745792388916  |
| 7 |   8.0   | 0.29545140266418457  |  3.197188377380371   |
| 8 |   9.0   | 0.007538318634033203 |  0.0038909912109375  |
| 9 |  10.0   |  0.244126558303833   |  0.6477646827697754  |
+---+---------+----------------------+----------------------+
